**Importing libraries**

In [24]:
pip install evaluate

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [25]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
from datasets import load_dataset, concatenate_datasets
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import wandb
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, pipeline
from evaluate import evaluator
import pandas as pd 
from datasets import Dataset, DatasetDict


**Loading data**

In [26]:
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("secret_val_0")
login(secret_value_0)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [27]:
dataset = load_dataset("OmidAghili/Sentiment_Analysis_Train-Validation")
testDataset = load_dataset("OmidAghili/Sentiment_Analysis_Test")

In [28]:
dataset=concatenate_datasets([dataset['train'],dataset['validation']]).shuffle(seed=42)
dataset

Dataset({
    features: ['reviewText', 'label'],
    num_rows: 380945
})

**Model**

In [29]:
model_name = "OmidAghili/Sentiment_Analysis_FinalModel"
pipe = pipeline("text-classification", model=model_name, device=0 )

In [30]:
texts = [
    "I love this product! It works perfectly.",
    "This is the worst purchase I have ever made.",
    "The quality is decent for the price.",
    "I am extremely satisfied with the service.",
    "The item broke after one use. Very disappointed."
]

                                
predictions = pipe(texts)

for text, pred in zip(texts, predictions):
    print(f"Text: {text}\nPrediction: {pred}\n")

Text: I love this product! It works perfectly.
Prediction: {'label': 'EXTREMELY POSITIVE', 'score': 0.9926884770393372}

Text: This is the worst purchase I have ever made.
Prediction: {'label': 'EXTREMELY NEGATIVE', 'score': 0.9885896444320679}

Text: The quality is decent for the price.
Prediction: {'label': 'NEURAL', 'score': 0.5144358277320862}

Text: I am extremely satisfied with the service.
Prediction: {'label': 'EXTREMELY POSITIVE', 'score': 0.9015859365463257}

Text: The item broke after one use. Very disappointed.
Prediction: {'label': 'EXTREMELY NEGATIVE', 'score': 0.875442385673523}



In [31]:
dataset = pd.DataFrame(dataset)

In [32]:
dataset.columns = ['text', 'label']

In [33]:
dataset = Dataset.from_pandas(dataset)

In [34]:
# dataset = dataset.select(range(100))  

In [35]:
task_evaluator = evaluator("text-classification")

eval_results = task_evaluator.compute(
    model_or_pipeline = pipe,
    data = dataset,
    label_mapping = {"EXTREMELY NEGATIVE": 0, "NEGATIVE": 1,"NEURAL":2,"POSITIVE":3, "EXTREMELY POSITIVE":4}
)

print(eval_results)

{'accuracy': 0.7012718371418446, 'total_time_in_seconds': 3262.207971365, 'samples_per_second': 116.77520358722005, 'latency_in_seconds': 0.008563461841906312}
